# Reinforcement Learning Note
1. 强化学习问题通常可以转化成马尔科夫决策流程`MDP`，由一组状态$\mathcal{S}、动作\mathcal{A}、概率\mathcal{P}、奖励\mathcal{R}、折扣因子\gamma$组成。
   1. $\mathcal{P}$表示不同转换和奖励发生的概率，通常建模为单个联合概率：任何时间步$t+1$的状态和奖励仅依赖于上一个时间步$t$的状态和采取的动作。
   2. 估算或预测的状态值$V(S)$和在特定状态下采取的动作的值$Q(S,A)$，后者用于判断应该采取什么动作，这两个映射或方法关系紧密，可以帮助我们找到问题的最优策略$\pi^*$，从而最大化接收的总奖励
   3. `MDP`本质上具有概率性，是无法完全确定的预测未来将获得什么样的奖励以及持续多久，因此计算的通常是总期望奖励
   4. 折扣因子$\gamma$：在计算状态和动作值时，用来给未来的奖励分配更低的权重
$$Framework
\begin{cases}
    MDP:(\mathcal{S}、\mathcal{A}、\mathcal{P}、\mathcal{R}、\gamma)& 
        \begin{cases}
        单个联合概率:\mathbb{P}(S_{t+1},R_{t+1}\vert S_t,A_t)\\
        状态值函数：V(S)\\
        动作值函数：Q(S,A)
        \end{cases}\\ \\
    Goal:& 找到问题最优策略\pi^*，最大化总期望奖励
\end{cases}
$$
2. 强化学习算法通常分为两个大类
   1. 基于模型的方法，例如策略迭代和需要已知转换和奖励模型的值迭代。本质上通过动态规划并使用该模型以迭代方式计算期望的值函数和最优策略
   2. 不基于模型的方法，例如蒙特卡洛方法和时间差分学习等。不需要明确的模型，通过执行探索性动作对环境抽样，并使用获得的经验直接估算值函数
$$Algorithm
\begin{cases}
基于模型的方法:& 
    \begin{cases}
    策略迭代\\
    值迭代
    \end{cases}\\
不基于模型的方法:& 
    \begin{cases}
    蒙特卡洛方法\\
    时间差分学习
    \end{cases}\\
\end{cases}
$$

---
## 一、强化学习框架：问题

---
### 1.强化学习框架（`RL框架`）
1. __RL框架__是指智能体（`Agent`）学习如何与环境互动。<br>假设时间会流逝并离散化时间步（`Time Steps`），在一开始的时间步中，只能体会观察环境，可以把这种结果看成环境呈现给智能体的情形，然后他必须选择合适的响应动作（`Action In Response`）。在下一个时间步对智能体的动作做出响应时，环境向智能体呈现新的情形，同时环境向智能体提供一个奖励，作为回应，智能体必须做出一个动作表示智能体对环境是否做出了正确的响应。这一流程继续下去，在每个时间步，环境都向智能体发送一个观察结果和奖励；作为回应，智能体必须选择一个动作。<br>通常，我们不需要假设环境向智能体显示做出合理决策所需的一切信息。但是如果这样假设，数学会大大简化，在这章节的课程中，假设智能体能够完全观察环境所处的任何状态。此时，智能体接受的不再是观察结果，而是环境状态。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/cfff4572-b080-48ab-8872-a96d45468761" alt="" width="500px" class="index--image--1wh9w">

---
### 2.阶段性任务（`Episodic Task`）与连续性任务（`Continuing Task`）
1. 在强化学习任务中，具有清晰结束点的任务被称为，阶段性任务。将从头到尾的一系列完整互动称为一个阶段，当一个阶段结束，智能体根据获得的奖励总量，判断自己的表现，然后重头再开始，不过会带有前一次的任务记忆，使得每一次的表现越来越好。经历足够多的次数的训练后，智能体将会得出一个获得奖励非常高的策略。
2. 相应的，将没有清晰结束点的任务称为，连续性任务。智能体一直存活，需要学习选择动作的最佳方式同时与环境不断互动，

---
### 3.奖励假设（`Reward Hypothsis`）
- 在机器学习中有个重要的定义假设，即智能体的目标始终可以描述为最大化期望累计奖励（`Maximizing Expected Cumulative Reward`），这就是奖励假设。

---
### 4.动作（`Action`）、目标（`Goal`）和奖励（`Reward`）
1. 动作 - 智能体在每一个时间步根据环境，判断得出的响应；
2. 状态 - 提供给智能体的环境信息，使其能作出合理的动作；
3. 目标 - 最大化累计奖励；
4. 奖励 - 环境对智能体做出的动作的反馈，表示智能体是否做出了正确的动作。

---
### 5.折扣回报（`Discounted Return`）
1. 更改目标，更关注近期奖励，而不是遥远的未来的奖励。在随机时间步$t$，定义一个折扣率（`Discount Rate`）$\gamma \in \lbrack 0,1\rbrack$，则回报/折扣回报：$$G_{t} = R_{t+1}+\gamma R_{t+2}+\gamma^{2}R_{t+3}+\gamma^{3}R_{t+4}+\ldots = \sum_{k=0}^\infty \gamma^{k}R_{t+k+1}$$
   - 如果$\gamma = 0$，智能体只关心最即时的奖励。
   - 如果$\gamma = 1$，回报没有折扣。
   - $\gamma$的值越大，智能体越关心遥远的未来。$\gamma$的值越小，折扣程度越大，在最极端的情况下，智能体只关心最即时的奖励。
2. 使用折扣是为了避免无限未来带来的不良影响。
3. 当智能体在随机时间步选择动作时，就要用到折扣，这样程序就会更关注于获得更早出现的奖励，而不是稍后出现并且可能性较低的奖励

---
### 6.马尔可夫决策流程（`MDP - Markov Decision Process`）
1. 一步动态特性（`One-Step Dynamic Feature`）
   - 在随机时间步$t$，智能体环境互动变成一系列的状态、动作和奖励。$$(S_0, A_0, R_1, S_1, A_1, \ldots, R_{t-1}, S_{t-1}, A_{t-1}, R_t, S_t, A_t)$$当环境在时间步$t+1$对智能体做出响应时，它只考虑上一个时间步$(S_t,A_t)$的状态和动作。尤其是，它不关心再上一个时间步呈现给智能体的状态。（<em>换句话说</em>，环境不考虑任何$\lbrace S_0,\ldots,S_{t-1}\rbrace$）。<br>并且，它不考虑智能体在上个时间步之前采取的动作。（<em>换句话说</em>，环境不考虑任何$\{ A_0, \ldots, A_{t-1}\}$）。<br>此外，智能体的表现如何，或收集了多少奖励，对环境选择如何对智能体做出响应没有影响。（<em>换句话说</em>，环境不考虑任何$\{ R_0, \ldots, R_t \}$）。<br>因此，我们可以通过指定以下设置完全定义环境如何决定状态和奖励$$p(s',r|s,a) \doteq \mathbb{P}(S_{t+1}=s', R_{t+1}=r|S_t = s, A_t=a)$$对于每个可能的$s', r, s, \text{and } a$。这些条件概率用于指定环境的<strong>一步动态特性</strong>。
2. MDP
   - 一组状态$\mathcal{S}$
      - 在阶段性任务中，我们使用$\mathcal{S}^{+}$表示所有状态集合，包括终止状态。
   - 一组动作$\mathcal{A}$
   - 一组奖励$\mathcal{R}$
   - 一步动态特性$$p(s',r|s,a) \doteq \mathbb{P}(S_{t+1}=s', R_{t+1}=r|S_t = s, A_t=a)$$
   - 折扣率$\gamma \in [0,1]$
   
<img src="./definition_MDP.jpg" width="900px">

---
## 二、强化学习框架：解决方案
可以将解决方案理解为：智能体为了实现目的必须学会的一系列动作

---
### 1.策略（`Policy`）
1. __确定性策略__是从$\pi: \mathcal{S}\to\mathcal{A}$的映射。对于每个状态$s\in\mathcal{S}$，它都生成智能体在状态$s$时将选择的动作$a\in\mathcal{A}$
2. __随机性策略__是从$\pi: \mathcal{S}\times\mathcal{A}\to [0,1]$的映射。对于每个状态$s\in\mathcal{S}$和动作$a\in\mathcal{A}$，它都生成智能体在状态$s$时选择动作$a$的概率。

---
### 2.状态值函数（`State-Value Function`）
1. 策略$\pi$的__状态值函数__表示为$v_{\pi}$。对于每个状态$s \in\mathcal{S}$，它都生成智能体从状态$s$开始，然后在所有时间步根据策略选择动作的预期回报。即$$v_\pi(s) \doteq \text{} \mathbb{E}_\pi[G_t|S_t=s]$$我们将$v_\pi(s)$称之为__在策略__$\pi$__下的状态__$s$__的值__。
2. 记法$\mathbb{E}_\pi[\cdot]$来自推荐<a target="_blank" href="http://go.udacity.com/rl-textbook">教科书</a>，其中$\mathbb{E}_\pi[\cdot]$定义为随机变量的预期值（假设智能体遵守策略$\pi$）。

<img src="./definition_status_value_function.jpg" width="700px">

---
### 3.贝尔曼方程-1（`Bellman Expectation Equation`）
1. 对于一般__MDP__，我们需要使用期望值，因为通常即时奖励和下个状态无法准确地预测。因为奖励和下个状态是根据__MDP__的一步动态特性选择的。在这种情况下，奖励$r$和下个状态$s'$是从（条件性）概率分布$p(s',r|s,a)$中抽取的，__贝尔曼预期方程（对于$v_{\pi}$）__表示了任何状态$s$对于_预期即时奖励和下个状态的预期_值的值：$$v_\pi(s) = \text{} \mathbb{E}_\pi[R_{t+1} + \gamma v_\pi(S_{t+1})|S_t =s]$$
<img src="./definition_bellman_expectation_equation.jpg" width="900px">
2. __计算期望值__
   1. 如果智能体的策略$\pi$是<strong>确定性策略</strong>，智能体在状态$s$选择动作$\pi(s)$，贝尔曼预期方程可以重写为两个变量 ($s'$和$r$）的和：$$v_\pi(s) = \text{} \sum_{s'\in\mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,\pi(s))(r+\gamma  v_\pi(s'))$$在这种情况下，我们将奖励和下个状态的折扣值之和$(r+\gamma  v_\pi(s'))$与相应的概率$p(s',r|s,\pi(s))$相乘，并将所有概率相加得出预期值。
   2. 如果智能体的策略$\pi$是<strong>随机性策略</strong>，智能体在状态$s$选择动作$a$的概率是$\pi(a|s)$，贝尔曼预期方程可以重写为三个变量（$s'$、$r$和$a$）的和：$$v_\pi(s) = \text{} \sum_{s'\in\mathcal{S}^+, r\in\mathcal{R},a\in\mathcal{A}(s)}\pi(a|s)p(s',r|s,a)(r+\gamma  v_\pi(s'))$$在这种情况下，我们将奖励和下个状态的折扣值之和$(r+\gamma  v_\pi(s'))$与相应的概率$\pi(a|s)p(s',r|s,a)$相乘，并将所有概率相加得出预期值。

---
### 4.最优策略（`Optimal Policy`）
1. 策略$\pi '$定义为优于或等同于策略$\pi$（仅在所有$s\in\mathcal{S}$时$v_{\pi'}(s) \geq v_\pi(s)$）。
2. __最优策略$\pi_*$__对于所有策略$\pi$满足$\pi_* \geq \pi$。最优策略肯定存在，但并不一定是唯一的。
3. __最佳策略__正是智能体寻找的策略，是__MDP__的解决方案和实现目标的最佳策略。
4. 所有最优策略都具有相同的状态值函数$v_*$，称为__最优状态值函数__。

---
### 5.动作值函数（`Action-Value Function`）
1. 动作函数是指状态动作对$s, \pi(s)$的值是智能体从状态$s$开始并采取动作$\pi(s)$，然后遵守策略$\pi$所获得的预期回报。即$$q_\pi(s,a) \doteq \mathbb{E}_\pi[G_t|S_t=s, A_t=a]$$我们将$q_\pi(s,a)$称之为__在状态$s$根据策略$\pi$采取动作$a$的值__（或者称之为__状态动作对__$s,a$的值）。
2. 所有最优策略具有相同的动作值函数$q_*$，称之为__最优动作值函数__。
3. 对于确定性策略$\pi,v_\pi(s) = q_\pi(s, \pi(s))$适用于所有$s \in \mathcal{S}$。
4. 智能体确定最优动作值函数$q_*$后，它可以为所有$s\in\mathcal{S}$设置$$\pi_*(s) = \arg\max_{a\in\mathcal{A}(s)} q_*(s,a)$$快速获得最优策略。<br>注意，必须确保$v_*(s) = \max_{a\in\mathcal{A}(s)} q_*(s,a)$。
5. 如果在某个状态$s\in\mathcal{S}$中，有多个动作$a\in\mathcal{A}(s)$可以最大化最优动作值函数，你可以通过向任何（最大化）状态分配任意大小的概率构建一个最优策略。只需确保根据该策略给不会最大化动作值函数的动作（对于特定状态）分配的概率是 0% 即可。

---
### 6.贝尔曼方程-2
1. $q_\pi$的贝尔曼预期方程是：
$$
\begin{align}
q_\pi(s,a) & = \text{} \mathbb{E}_\pi[R_{t+1} + \gamma q_\pi(S_{t+1},A_{t+1}) |S_t=s,A_t=a]\\
 & = \sum_{s' \in \mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r + \gamma\sum_{a' \in \mathcal{A}(s)} \pi(a'|s') q_\pi(s',a'))\\
 & = \sum_{s'\in\mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r+\gamma v_\pi(s'))
\end{align}
$$
其中最后一个形式详细介绍了如何计算任意随机策略$\pi$的预期值。该方程表示任何_状态动作对_（根据任意策略）相对于_后续状态_的值（根据同一策略）的值。
2. 贝尔曼最优性方程：<br>和贝尔曼预期方程相似，贝尔曼最优性方程可以证明：状态值（以及动作值函数）满足递归关系，可以将状态值（或状态动作对的值）与所有后续状态（或状态动作对）的值联系起来。侧重于最优_策略对应的值满足的关系。
   1. __$v_*$的贝尔曼最优性方程__
   $$
   \begin{align}
   v_*(s)  & = \max_{a \in \mathcal{A}(s)} \mathbb{E}[R_{t+1} + \gamma v_*(S_{t+1}) | S_t=s]\\
    & = \max_{a \in \mathcal{A}(s)}\sum_{s' \in \mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r + \gamma v_*(s'))
    \end{align}
   $$
   它表示任何_状态_根据最优策略相对于_后续状态_的值（根据最优策略）的值。
   2. __$q_*$的贝尔曼最优性方程__
   $$
   \begin{align}
   q_*(s,a)  & = \mathbb{E}[R_{t+1} + \gamma \max_{a'\in\mathcal{A}(S_{t+1})}q_*(S_{t+1},a') | S_t=s, A_t=a]\\
    & = \sum_{s' \in \mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r + \gamma \max_{a'\in\mathcal{A}(s')}q_*(s',a'))
    \end{align}
   $$
   它表示任何_状态动作对_根据最优策略相对于_后续状态动作对_（根据最优策略）的值的值。

---
## 三.动态规划（`Dynamic Programming`）

---
### 1.迭代方法（`Iterative Method`）
1. 为了获得策略$\pi$对应的状态值函数$v_{\pi}$，我们只需求解$v_{\pi}$的贝尔曼预期方程对应的方程组。
2. 虽然可以通过分析方式求解方程组，但是通常状态空间非常大，使得直接求解方程组变得非常困难。使用迭代方法简化这一过程。
3. 先猜测每个状态的值（通常可以设为0），然后关注某一个状态，再利用状态之间的关系以及贝尔曼期望方程，更新关注的那个状态的值。如此循环下去...

---
### 2.迭代策略评估算法（`Iterative Policy Evalution Algorithm`）
- __迭代策略评估__是在动态规划设置中用到的算法，用于估算策略$\pi$对应的状态值函数$v_{\pi}$。在此方法中，我们将向值函数估值中不断循环应用贝尔曼更新，直到估值的变化几乎不易觉察。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/969aeff0-ee0e-4ac3-a0bf-f933f2f55154" alt="" width="427px" class="index--image--1wh9w">

---
### 3.动作值的估算
1. 在动态规划设置中，可以使用以下方程从状态值函数$v_{\pi}$快速获得动作值函数$q_{\pi}$：$$q_\pi(s,a) = \sum_{s'\in\mathcal{S}, r\in\mathcal{R}}p(s',r|s,a)(r+\gamma v_\pi(s'))$$<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/e2f11231-1c11-4f33-b1ce-17bdfa797617" alt="" width="507px" class="index--image--1wh9w">
2. 在__确定性__环境中，智能体选择某个动作后，下个状态和奖励可以 100% 确定不是随机的。对于确定性环境，所有的$s', r, s, a$为$p(s',r|s,a) \in \{ 0,1 \}$。在这种情况下，当智能体处在状态$s$并采取动作$a$ 时，下个状态$s'$和奖励$r$可以确切地预测，我们必须确保$q_\pi(s,a) = r + \gamma v_\pi(s')$
3. 在__随机性__环境中，智能体选择动作后，下个状态和奖励无法确切地预测，而是从<a target="_blank" href="https://en.wikipedia.org/wiki/Conditional_probability_distribution">（条件性）概率分布</a>$p(s',r|s,a)$中随机抽取的。在这种情况下，当智能体处在状态$s$并采取动作$a$时，每个潜在下个状态$s'$的概率和奖励$r$由$p(s',r|s,a)$确定。在这种情况下，我们必须确保$$q_\pi(s,a) = \sum_{s'\in\mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r+\gamma v_\pi(s'))$$，我们计算和$r + \gamma v_\pi(s')$的<a target="_blank" href="https://en.wikipedia.org/wiki/Expected_value">期望值</a>

---
### 4.策略改进（`Policy Improvement`）
1. 该算法使用策略的值函数，提出一个至少和当前策略一样好的新策略。
2. __策略改进__对策略$\pi$对应的动作值函数$v_{\pi}$进行估算$V$，并返回改进（或对等）的策略$\pi'$，其中$\pi'\geq\pi$。该算法首先构建动作值函数估值$Q$。然后，对应每个状态$s\in\mathcal{S}$，你只需选择最大化$Q(s,a)$的动作$a$。换句话说，$\pi'(s) = \arg\max_{a\in\mathcal{A}(s)}Q(s,a)$，针对所有。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/e3fbf474-e22f-4bcd-b5e7-0c2db9240c24" alt="" width="531px" class="index--image--1wh9w">
3. 实现过程：
   1. 将值函数转换成动作值函数
   2. 对每个状态，选择最大化动作值函数的动作
   3. 对于某个状态下，有多个动作可以最大化动作值函数的情况
      1. 可以随机选择一个动作
      2. 或者构建一个随机性策略，为任何/所有动作分配非零概率

---
### 5.策略迭代（`Policy Iteration`）
1. 该算法先对最优策略进行猜测，从对等随机策略（对于每个状态，选择每个动作的概率都是一样的）开始比较合适，通过策略评估获得相应的值函数，再使用该值函数和策略改进，获得一个潜在完善的新策略，然后带入该新策略，再次进行策略评估，然后进行策略改进，不断重复下去，直到收敛于最优策略。<img src="policy_iteration.jpg" width="800px">
2. __策略迭代__是一种可以在动态规划设置中解决 MDP 的算法。它包含一系列的策略评估和改进步骤，肯定会收敛于最优策略（对应任意_有限_ MDP）。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/c7ba887a-451a-42b6-8283-cb5d7450f4e6" alt="" width="475px" class="index--image--1wh9w">

---
### 6.截断策略迭代（`Truncated Policy Evaluation`）
1. 策略迭代花费的时间，与设置的极小数$\theta$（$\theta$越小，时间越长）以及`MDP`的具体情况相关。通过修改终止条件，使得在对状态空间执行固定次数的遍历后，停止评估步骤,此方法称为__截断策略评估__。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/331504f7-cbef-4d0f-b919-5c26bf7ff707" alt="" width="474px" class="index--image--1wh9w">
2. 将这个修订后的策略评估算法应用到类似于策略评估的算法中，称之为__截断策略迭代__。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/a050158f-1833-4ff7-b51c-4af3868965b3" alt="" width="473px" class="index--image--1wh9w">
3. 截断策略迭代的停止条件与策略迭代的不同。在策略迭代中，当策略在一次策略改进步骤之后没有变化时，我们将终止循环。在截断策略迭代中，仅当值函数估值收敛时，我们才停止循环。<img src="truncated_policy_iteration.jpg" width="800px">

---
### 7.值迭代（`Value Iteration`）
1. __值迭代__是在动态规划设置中用来估算策略$\pi$对应的状态值函数$v_\pi$的算法。对于此方法，每次对状态空间进行遍历时，都同时进行策略评估和策略改进。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/6c0e0ed5-f481-487b-8ff3-159051fce0d2" alt="" width="432px" class="index--image--1wh9w">
2. 如果后续值函数估值之间的差值很小，则满足了停止条件。尤其是，如果对于每个状态，差值都小于$\theta$，则循环终止。并且，如果我们希望最终值函数估值与最优值函数越接近，则需要将值$\theta$设得越小。

---
## 四.蒙特卡洛方法（`Monte_Carlo Method`）
智能体以阶段形式与环境互动，一个阶段结束后，按顺序查看每个状态动作对，如果是首次经历，则计算相应的回报并使用它来更新动作值，经历很多很多个阶段。主要注意的是：只要不在阶段之间更改策略，该算法就可以解决预测问题。只要运行概算足够长的时间，就肯定能获得一个很完美的动作值函数估计结果<img src="monte_carlo_method.jpg" width="800px">

---
### 1.MC预测：状态值
1. 解决预测问题的算法会确定策$\pi$对应的值函数$v_\pi$（或$q_\pi$）。
2. 通过与环境互动评估策略$\pi$的方法分为两大类别：
   1. <span id="jump_to_offpm">__离线策略方法（`Off-Policy Method`）__使智能体与环境互动时遵守的策略$b$（其中$b\neq\pi$）与要评估（或改进）的策略不同。</span>
   2. <span id="jump_to_onpm">__异同策略方法（`On-Policy Method`）__使智能体与环境互动时遵守的策略$\pi$与要评估（或改进）的策略相同。</span>
3. 状态值是该状态之后的预期回报，因此智能体体验的平均回报是个很好的估值。
4. 状态$s\in\mathcal{S}$在某个阶段中的每次出现称为$s$的一次经历。
5. 有两种类型的蒙特卡洛 (`MC`) 预测方法（用于估算$v_\pi$）：
   1. __首次经历 MC（`First-Visit MC Method`）__ 将$v_\pi(s)$估算为仅在$s$首次经历之后的平均回报（即忽略与后续经历相关的回报）。
   2. __所有经历 MC（`Every-Visit MC Method`）__ 将$v_\pi(s)$估算为$s$所有经历之后的平均回报。
   3. 两者的差别在于：
      1. 所有经历 MC 存在<a target="_blank" href="https://en.wikipedia.org/wiki/Bias_of_an_estimator">偏差</a>，而首次经历 MC 不存在偏差。
      2. 一开始，所有经历 MC 具有更低的<a target="_blank" href="https://en.wikipedia.org/wiki/Mean_squared_error">均方误差 (MSE)</a>，但是随着经历更多的阶段，首次经历 MC 的均方误差更低。
   4. 当每个状态的经历次数接近无穷大时，首次经历和所有经历方法都__保证会收敛于__真值函数。（换句话说，只要智能体在每个状态获取足够的经验，值函数估值将非常接近真值。）
   
<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/c46f7b7a-8343-4a21-a6e1-9dd8f144d3a5" width="602px">

---
### 2.MC预测：动作值
1. 在强化学习设置下并不知道动态特性，为了获得动作值，不再去查看每个状态的经历，而是查看每个潜在动作对的经历，计算每个动作状态对之后的回报，取平均值
2. 状态动作对$s,a (s\in\mathcal{S},a\in\mathcal{A})$在某个阶段中的每次出现称为$s,a$的一次经历。
3. 有两种类型的蒙特卡洛 (`MC`) 预测方法（用于估算$q_\pi$）：
   1. __首次经历`MC`__将$q_\pi(s,a)$估算为仅在$s,a$首次经历之后的平均回报（即忽略与后续经历相关的回报）。
   2. __所有经历`MC`__将$q_\pi(s,a)$估算为$s,a$所有经历之后的平均回报。
   
<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/0d911b4b-550c-494d-8324-31f703a93c26" width="602px">

---
### 3.广义策略评估（`Generalized Policy Iteration`）
指：不对策略评估周期次数进行限制，并且不对收敛接近程度进行限制的算法。在所有强化学习下都可以使用。

---
### 4.MC控制：增量均值
1. 假设一个动作状态对经历了一定的次数，将回报表示为$\lbrace x_1, x_2, \ldots, x_n\rbrace$，然后通过对这些值取均值，计算值函数逼近结果，表示为：$$\mu_n = \frac{\sum_{j=1}^{n}x_j}{n}$$
2. 在每次经历之后都迭代更新估值，通过推到可以得出第$k$的均值与第$k-1$的均值的关系：
$$
\begin{align}
\mu_k & = \frac{1}{k}\sum_{j=1}^{k}x_j\\
 & = \frac{1}{k}(x_k + \sum_{j=1}^{k-1}x_j)\\
 & = \frac{1}{k}(x_k + (k-1)\mu_{(k-1)})\\
 & = \mu_{(k-1)} + \frac{1}{k}(x_k - \mu_{(k-1)})
\end{align}
$$
<img src="incremental_mean.jpg" width="800px">

---
### 5.MC控制：策略评估
1. 通过对增量均值进行改进，使得能获得多个状态动作对的值函数逼近结果。
2. 智能体先从环境中获得一个阶段样例，然后对于每个时间步，查看相应的状态动作对，如果是首次访问，则计算相应的回报，然后根据不断取平均值的算法，更新动作值的相应估值；初始化每个对的经历次数，将上述计算带入蒙特卡洛控制计算中。
<img src="mc_policy_evaluation.jpg" width="800px">

---
### 6.MC控制：策略改进
1. 如果对于每个状态$s\in\mathcal{S}$，它保证会选择满足$a = \arg\max_{a\in\mathcal{A}(s)}Q(s,a)$的动作$a\in\mathcal{A}(s)$，则策略相对于动作值函数估值$Q$来说是贪婪策略。（通常将所选动作称之为贪婪动作。）
2. 如果对于每个状态$s\in\mathcal{S}$，策略相对于动作值函数估值$Q$是$\epsilon$贪婪策略。
   1. 概率为$1-\epsilon$时，智能体选择贪婪动作，以及
   2. 概率为$\epsilon$时，智能体随机（均匀地）选择一个动作。

---
### 7.Epsilon 贪婪策略
1. 为了构建一个相对于当前动作值函数估值$Q$为$\epsilon$贪婪策略的策略$\pi$，我们只需设置<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/cd71ea2c-3df6-4012-b65a-d11a90e60632" width="600px">针对每个$s\in\mathcal{S}$和$a\in\mathcal{A}$。注意，$\epsilon$必须始终是 0-1（含）之间的值（即$\epsilon \in [0,1]$）。
2. 可以将遵守$\epsilon$贪婪策略的智能体看做始终可以操控硬币方向（可能不公平），正面朝上的概率是$\epsilon$。观察状态后，智能体就会抛掷该硬币。
   1. 如果硬币背面朝上（因此概率为$1-\epsilon$），智能体选择贪婪动作。
   2. 如果硬币正面朝上（因此概率为$\epsilon$），智能体从一组潜在（非贪婪和贪婪）动作中均匀地随机选择一个动作。

---
### 8.探索-利用
1. 探索-利用困境
   - 注意，智能体一开始不知道环境的一步动态特性。为了最大化回报，智能体必须通过互动了解环境。<br>在每个时间步，当智能体选择一个动作时，它都根据以前与环境的互动经验做出决策。并且，为了尽量减少求解`OpenAI Gym`中的环境所需的阶段次数，我们的第一个想法是设计一种策略，其中智能体始终选择它认为（根据过去的经验）将最大化回报的动作。因此，智能体可以遵守对动作值函数估算来说很贪婪的策略。<br>为了了解为何是这种情况，注意在早期阶段，智能体所了解的信息很有限（可能有缺陷）。因此很有可能智能体_估计非贪婪的动作实际上比估计_贪婪的动作更好。<br>因此，成功的强化学习智能体不能在每个时间步都采取贪婪的动作（即它不能始终利用自己的经验）；为了发现最优策略，它必须不断改进所有状态动作对的估算回报（换句话说，它必须继续通过经历每个状态动作对探索各种可能性）。但是，智能体应该始终保持一定的贪婪性，从而_尽快_最大化回报。这就引出了$\epsilon$贪婪策略。<br>我们将需要平衡这两个竞争性条件的情况称为探索-利用困境。解决该困境的一个潜在方法是在构建$\epsilon$贪婪策略时逐步修改$\epsilon$的值。
2. 设置$\epsilon$的值
   - 合理的做法是智能体一开始与环境互动时，倾向于探索环境，而不是利用已有的经验。毕竟当智能体对环境的动态特性相对知之甚少时，应该怀疑自己的有限知识并探索环境，或者尝试各种最大化回报的策略。因此，最佳起始策略是等概率随机策略，因为它在每个状态探索所有潜在动作的概率是相同的。你在上个练习中发现，$\epsilon = 1$会生成一个等同于等概率随机策略的$\epsilon$贪婪策略。<br>在后续时间步，合理的做法是倾向于利用已有的经验，而不是探索环境，策略在动作值函数估算方面越来越贪婪。毕竟智能体与环境的互动次数越多，它对估算动作值函数的信任就越高。你在上个练习中发现，$\epsilon = 0$会生成贪婪策略（或者非常倾向于利用已有的经验而不是探索环境的策略）。<br>幸运的是，此策略（一开始倾向于探索环境而不是利用已有的经验，然后逐渐倾向于利用已有的经验而不是探索环境）可以证明是最优策略。
3. 有限状态下的无限探索贪婪算法 (`GLIE`)
为了保证 MC 控制会收敛于最优策略$\pi_*$，我们需要确保满足两个条件。我们将这些条件称之为__有限状态下的无限探索贪婪算法 (GLIE)__。尤其是，如果：
   - 每个状态动作对$s, a$（针对所有$s\in\mathcal{S}$和$a\in\mathcal{A}$）被经历无限次;
   - 策略收敛相对于动作值函数估算 QQ 来说贪婪的策略。
然后，MC 控制保证会收敛于最优策略（在有限状态下运行无穷个阶段）。这些条件确保：
   - 智能体继续在所有时间步进行探索
   - 智能体逐渐倾向于利用已有的经验（而不是探索环境）。
满足这些条件的一种方式是在指定$\epsilon$贪婪策略时，修改$\epsilon$的值。尤其是，使$\epsilon_i$对应于第$i$个时间步。然后在以下情况下，这两个条件都会满足：
   - 对于所有时间步$i，\epsilon_i > 0$
   - 当时间步$i$接近无穷大时，$\epsilon_i$减小到0（即$\lim_{i\to\infty} \epsilon_i = 0$）。
<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/2eba43ac-3b4f-470a-8783-a0c80277c16f" alt="" width="540px" class="index--image--1wh9w">

---
### 9.MC控制：常量$\alpha$
1. 综合前述算法，生成一个阶段，然后对于经历的每个状态动作对$(S_t, A_t)$，都计算接下来的相应回报$G_t$（`Corresponding Return`）,然后再用该回报获取最新的估算，更新步骤表示如下：$$Q(S_t, A_t) \longleftarrow Q(S_t, A_t) + \frac{1}{N(S_t,A_t)}(G_t - Q(S_t, A_t))$$
2. 对于上述算法，可以这样理解：
   1. 计算最近取样的回报$G_t$和对应的状态动作对$(S_t, A_t)$之间的差别，记作：$\delta_t = G_t - Q(S_t, A_t)$
   2. $\delta_t$是期望回报$Q(S_t, A_t)$与实际回报$G_t$之间的差值
      1. $\delta_t > 0$，实际回报比值函数期望回报要大，说明值函数偏小，因此需要增大估值$Q(S_t, A_t)\uparrow$
      2. $\delta_t < 0$，实际回报比值函数期望回报要小，说明值函数偏大，因此需要降低预期的动作值函数$Q(S_t, A_t)\downarrow$
      3. 增大/降低的调整幅度与已经经历状态动作对的次数$N(S_t,A_t)$成反比
      4. 这种方式，前面的回报对调整影响大，后面的影响小。因此我们需要更改该算法。
3. 引入步长参数$\alpha$，并更新算法：<span id="jump">$$Q(S_t, A_t) \longleftarrow Q(S_t, A_t) + \alpha(G_t - Q(S_t, A_t))$$</span>
   1. 这样可以确保稍后获得的回报比之前获得的回报更受重视，这样，智能体将最信任最新的汇报，并逐渐忘记先前获得的回报。这点非常重要，因为该策略不断改变，每一步都越来越优化，因此实际上后面的时间步对估算动作值来说很重要
   2. 步长参数$\alpha$必须满足$0 < \alpha \leq 1$。$\alpha$值越大，学习速度越快，但是如果$\alpha$的值过大，可能会导致__MC__控制无法收敛于$\pi_*$。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/4c12506d-c61a-4c54-8f06-18e65940cae0" alt="" width="627px" class="index--image--1wh9w">

---
## 五.时间差分方法（`Temporal Difference Learning`）
学习如何认识用时间差分法（例如Sarsa、Q学习和预期）解决阶段性任务和连续性任务<img src="td_control_algorithms.jpg" width="800px">

---
### 1.TD 预测：TD(0)
1. 将[四.9.3的数学式变形：](#jump)$$V(S_t) \longleftarrow V(S_t) + \alpha(G_t - V(S_t))$$根据：
$$
\begin{align}
v_\pi(s) & = \text{} \mathbb{E}_\pi[G_t|S_t=s]\\
 & = \text{} \mathbb{E}_\pi[R_{t+1} + \gamma V_(S_{t+1})|S_t =s]
\end{align}
$$代入上式，变形得：<span id="jump1">$$V(S_t) \longleftarrow V(S_t) + \alpha([R_{t+1} + \gamma V_(S_{t+1})] - V(S_t))$$</span>任何状态的值定义为智能体遵守策略后在该状态之后很可能会出现的预期回报。对取样回报取平均值生成了很好的估值，
2. 对于上式可以这样理解：
   1. 先查看时间$t$和$t+1$处状态的值，带入奖励$R_{t+1}$，得出右式的值
   2. 将这个值作为时间$t$时的状态的新估算值赋给$V(S_t)$
   3. 这样，我们不用等待阶段结束，就可以更新值
   4. 这是一个处理连续性问题时的预测问题算法
<img src="td(0).jpg" width="800px">
3. 右式中，当前状态的最佳估算值$V(S_t)$，智能体采取动作并获得奖励$R_{t+1}$和下个状态$V(S_{t+1})$，使用这些后续状态的信息表达同一状态$S_t$的值替代估值，并将新的估值$R_{t+1} + \gamma V_(S_{t+1})$记为__TD目标（TD Target）__
   - 这个更新方程的作用是：找到两个估值$V(S_t)$（`Previous Estimate`）$\&$ $R_{t+1} + \gamma V_(S_{t+1})$（`TD target`）之间的中间值
4. 将数学式合并同类项后得：$$V(S_t) \longleftarrow (1 - \alpha)V(S_t) + \alpha(R_{t+1} + \gamma V_(S_{t+1}))$$可见$\alpha$是介于0-1之间的数
   1. 当$\alpha = 1$，新估值就是`TD target`
   2. 当$\alpha = 0$，新估值完全忽略`TD target`，只保留旧估值
   3. 上述两种都不利于智能体的学习，通常把$\alpha$设置为一个接近0的小值更有帮助。$\alpha$越小，对`TD target`的信任就越低，并更加依赖于状态值的现有估值。
5. 将这个更新步骤放入完整的算法中，称之为一步时间差分（`One-Step Temporal Difference/TD`）
   1. 一步是指在每个时间步之后都更新值函数，也称之为__TD(0)__
   2. 用于确认对应于某个策略$\pi$的状态值函数
      1. 对于连续性问题
         1. 先将每个状态初始化为0
         2. 在每个时间步，智能体都与环境互动，选择由策略决定的动作
         3. 在环境中获得奖励和下个状态后，立即更新上个状态的值函数
      2. 对于阶段性任务
         1. 对于每个阶段，只需要检查在每个时间步，最近的状态是否为最终状态
         2. 循环运行更新步骤更新上一个状态直到$S_t$是最终状态
         3. 然后对下一个阶段运行步骤1，2
   3. 只要互动足够长或者生成的阶段足够多，该算法就应该会返回一个很好的值函数逼近结果<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/021fe61b-6e90-441a-b17e-1c0d549bd5d1" alt="" width="630px" class="index--image--1wh9w">
   4. 相比于常量$\alpha$MC预测，TD(0)有以下优点：
      1. 虽然 MC 预测必须等到阶段结束时才能更新值函数估值，但是， TD 预测方法在每个时间步之后都会更新值函数。同样，TD 预测方法适合连续性和阶段性任务，而 MC 预测只能应用于阶段性任务。
      2. 在实践中，TD 预测的收敛速度比 MC 预测的快。（但是，没有人能够证明这一点，依然是一个需要验证的问题。）

---
### 2.TD预测：动作值
- 调整[1](#jump1)中的流程，使之返回动作值的估值：$$Q(S_t,A_t) \longleftarrow Q(S_t,A_t) + \alpha([R_{t+1} + \gamma Q(S_{t+1},A_{t+1})] - Q(S_t,A_t))$$唯一的区别就在于：智能体在每次选择动作后更新值，如果智能体与环境互动足够长的时间，将能够准确的估算动作值函数

---
### <span id="jump_to_sarsa">3.TD控制：Sarsa</span>
1. 该算法用于确定最优策略
2. Sarsa算法具体步骤：
   1. 在每个时间步使用一个针对当前动作估值的Epsilon贪婪策略选择一个动作，在初始时间步，先将$\epsilon$设为1
   2. 根据对等概率随机策略（`Equal Probable Random Policy `）选择$A_0和A_1$
   3. 对选顶动作之后的所有未来时间步，都更新动作值函数并构建相应的Epsilon贪婪策略
   4. 只要为$\epsilon$指定合适的值，该算法就肯定会收敛于最优策略<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/b97fee66-b767-437c-8a18-bc5712b7f7fd" alt="" width="619px" class="index--image--1wh9w">
3. 每个动作只更新都使用状态动作奖励、后续状态、后续动作、互动元组
4. 只要步长参数$\alpha$足够小，并且满足有限状态下的无限探索贪婪算法 (GLIE) 条件，Sarsa(0) __保证会收敛于__最优动作值函数。虽然有满足 GLIE 条件的很多方式，但是有一种方式在构建$\epsilon$贪婪策略时会逐渐降低$\epsilon$的值。尤其是，使$\epsilon_i$ 对应于第 ii 个时间步。然后，如果我们按照以下条件设置$\epsilon_i$：
   1. 对于所有时间步$i，\epsilon_i > 0$，以及
   2. 当时间步$i$接近无穷大时，$\epsilon_i$降低到 0（即$\lim_{i\to\infty} \epsilon_i = 0$），
   
   然后该算法会保证产生一个很好的$q_*$ 估值，只要我们运行该算法足够长的时间。然后，可以通过对所有$s\in\mathcal{S}$设置$\pi_*(s) = \arg\max_{a\in\mathcal{A}(s)} q_*(s, a)$，获得相应的最优策略$\pi_*$。

---
### <span id="jump_to_sarsamax">4.TD控制：Sarsamax（aka Q-Learning）</span>
1. 与Sarsa相比，采用相同的初始动作值$Q(S_0, A_0)$和策略$\pi$，智能体接收初始状态$S_0$，依然根据初始策略选择第一个动作$A_0$，接受奖励$R_0$和下一个状态$S_1$，然后在选择下一个动作之前使用贪婪动作更新动作值$$Q(S_0,A_0) \longleftarrow Q(S_0,A_0) + \alpha([R_{1} + \gamma \max_{a \in A}Q(S_{1},a)] - Q(S_0,A_0))$$和策略$$\pi \longleftarrow \epsilon-greedy(Q)$$再使用刚更新的动作值对应的Epsilon贪婪策略选择$A_1$，收到奖励和下个状态后继续这一流程，像之前使用贪婪动作更新$(S_1,A_1)$对应的动作一样，使用相应的Epsilon贪婪策略选择$A_2$，以此类推$$Q(S_t,A_t) \longleftarrow Q(S_t,A_t) + \alpha([R_{t+1} + \gamma \max_{a \in A}Q(S_{t+1},a)] - Q(S_t,A_t))$$
2. 与[Sarsa](#jump_to_sarsa)对比：
   1. 在Sarsa中更新步骤使动作值更接近于智能体当前遵守Epsilon贪婪策略获得的动作值
   2. Sarsamax直接在每个时间步估算最优值函数

<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/c9163e1d-7f4c-47a4-8a8b-f3af15f6bb7a" alt="" width="617px" class="index--image--1wh9w">

---
### 5.TD控制：Sarsa期望（`Expected Sarsa`）
- 与[Sarsamax](#jump_to_sarsamax)带入最大化下个状态对应的动作估值选择在此处采取的值$\gamma \max_{a \in A}Q(S_{t+1},a)$不同，期望Sarsa使用下个状态动作对的期望值$\gamma \sum_{a \in A}\pi(a\bracevert S_{t+1})Q(S_{t+1},a) - Q(S_t,A_t)$，期望值会考虑智能体从下个状态选则每个可能的动作的概率<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/9fbb47e8-78af-41ff-9329-5f89bb753142" alt="" width="617px" class="index--image--1wh9w">

---
### 6.分析性能
1. 在以下情况下，我们讨论过的所有`TD`控制算法（Sarsa、Sarsamax、预期 Sarsa）都会收敛于最优动作值函数$q_*$（并生成最优策略$\pi_*$）：
   1. $\epsilon$的值根据__GLIE__条件逐渐降低
   2.  步长参数$\alpha$足够小
2. 这些算法之间的区别总结如下：
   1. Sarsa 和预期 Sarsa 都是[__异同策略__ ](#jump_to_onpm)TD 控制算法。在这种情况下，我们会根据要评估和改进的相同（$\epsilon$贪婪策略）策略选择动作。
   2. Sarsamax 是[__离线策略__](#jump_to_offpm)方法，我们会评估和改进（$\epsilon$贪婪）策略，并根据另一个策略选择动作。
   3. 既定策略 TD 控制方法（例如预期 Sarsa 和 Sarsa）的在线效果比新策略 TD 控制方法（例如 Sarsamax）的要好。
   4. 预期 Sarsa 通常效果比 Sarsa 的要好。

---
## 六.深度强化学习（`Deep Reinforcement Learning`）
1. 深度强化学习指：使用深度学习方法来解决强化学习问题。
2. 强化学习通常包括有限的`MDP`，即状态和动作数量是有限的。但是有太多问题具有非常大的状态和动作空间，甚至由连续的实数组成，传统算法使用表格或字典或其他有限结构来记录状态和动作值无法解决此类问题。因此通过深度强化学习
   1. 泛化已有算法以便适合大型连续空间
   2. 基于值的技巧，如深度Q学习`Deep Q Learning`
   3. 直接尝试优化策略的方法，如策略梯度`Policy Gradients`
   4. 行动者-评论者方法`Actor-Critic Methods`

---
### 1.离散空间（`Discrete Space`）和连续空间（`Continuous Space`）
1. 离散空间
   1. 假设任何时间的环境状态来自于一组潜在状态，称之为离散状态空间。动作也类似，如果有一组有限的动作，则表示环境有一个离散动作空间
2. 连续空间
   1. 连续空间中的量值是一定范围的值，这个范围通常是实数
   2. 连续空间之所以重要，是因为真实世界的问题往往无法用网格话来建立模型，也就不可能是离散空间。
   3. 为了能处理连续空间问题，使用离散化（`Discretization`）和函数逼近（`Function Approximation`）策略

---
### 2.离散化（`Discretization`）
1. 离散化就是将__连续空间__使用区域划分的方法划分成__离散空间__。主要的方法有`Tile Coding`、`Coarse Coding`。并且__状态空间可以被不均匀地细分__。
2. 虽然底层环境是连续的，但是在__离散化__状态空间表示法中，仅需要某些位置标识信息。虽然在确定坐标时使用四舍五入的处理会导致位置不准确，但对某些环境，离散化状态空间效果很好，可以几乎不加修改就可以使用现有的算法。
3. 动作也可以离散化，例如：角可以拆分为完整度数。
4. 对于网格表示法，当物体超出单元格可以表示的精度，可以通过再拆分分单元格的方式提升精度。
5. `Tile Coding`和`Coarse Coding`都是在随机环境中均能使用的离散化方法（_下面本该有一个位和位向量的概念，但是我无法理解这两个概念，所以先回避这俩_）
   1. `Tile Coding`
      1. 底层状态空间是连续的二维空间，在该空间上方叠加多个网格或拼贴每一层稍微不对齐。状态空间里的任何位置$\mathcal{s}$，都可以粗略的通过它所激活的拼贴识别<img src="tile_coding.jpg" width="80%"><img src="tile_coding_algorithm.jpg" width="80%">
      2. 由于Tile Coding需要提前手动选择拼贴大小、偏移量、拼贴数量等等，因此就有了更灵活的方法`Adaptive Tile Coding`。
         1. 先从非常大的拼贴开始，然后在合适的时候将每个拼贴一分为二。
         2. 通过试探法，来决定何时拆分单元格，即当前不再学到很多规律时，也就是值函数不再改变时，就需要拆分状态空间；达到拼贴数量上限或迭代上限时，停止拆分。
         3. 跟踪子拼贴以及它们的预测权重，选择子拼贴之间权重差别最大的拼贴，这个拼贴就是对值函数影响最大的，也就是需要被拆分的拼贴。
         4. `Adaptive Tile Coding`的优势在于：不需要手动提前指定离散化方式，最终得到一个基于空间复杂程度的状态空间的划分
   2. `Coarse Coding`
      1. 与`Tile Coding`很相似，但是使用一组更稀疏的特征来表示状态空间
      2. 在二位连续状态空间上放下一堆圆圈，选择任何一个状态$\mathcal{s}$即在该空间中的位置，并标记它所属的所有圆圈，这些圆圈用`1`表示，剩余圆圈用`0`表示，这就是状态的稀疏编码表示法。这个方法可以拓展到更高维度的空间，圆圈变成球面和超球面。
      3. `Coarse Coding`的一些特性：
         1. 圆圈越小，空间的泛化程度越低，算法需要持续时间越长，但是可以获得更有效的分辨率
         2. 圆圈越大，泛化程度越高，通常会形成更平缓的值函数
         3. 可以使用更少的大圆圈来涵盖空间，从而缩小表示法，但是会损失一定的分辨率
         4. 不仅可以改变圆圈的大小，还可以改变他们的宽度、高度，甚至是任何形状，以便在一个维度里获得更高的分辨率<img src="coarse_coding.jpg" width="80%">
      4. 根据状态$\mathcal{s}$到每个圆圈中心的距离衡量该特征的活跃程度，可以用高斯或钟形曲线，是这种衡量方式或响应更加平缓，曲线在圆圈上居中，称之为径向基函数$$\phi_i(s)\doteq exp\left( -\frac{\Arrowvert s-c_i\Arrowvert^2}{2\sigma_i^2}\right)$$<img src="radial_basis_functions.jpg" width="80%">形成的状态值不再离散化，因此又是另一个连续状态向量，但是可以显著降低特征数量。
6. 离散化的缺陷在于：
   1. 当底层空间很复杂时，所需的离散状态数量可能会很大
   2. 对于状态空间中邻近的位置，他们的值应该相似或者平滑地变化，但离散化并非始终会利用这一特性，无法跨空间的有效泛化

---
### 3.函数逼近（`Function Approximation`）
1. 对于那么很复杂的问题，无法得知真正的底层函数，定义此类函数逼近的一般方法是引入一个表示函数形状的参数向量$W$，通过调整这个参数向量，找到理想的逼近结果$$\hat{v}(s,W) \approx v_\pi (s)\\ \hat{q}(s,a,W) \approx q_\pi (s,a)$$
2. 逼近函数可以将状态映射到其值或将状态动作对映射到相应的q值，另一种形式是从一个状态映射到一堆不同的q值同时映射到每个动作<img src="function_approximation.jpg" width="80%">
3. 以第一种情况（将状态映射到其值）为例，定义一个转换算法，将向量形式的给定状态$s$转换成特征向量$X(s)$，这样就避免直接运算原始状态。然后将特征向量$X(s)$与参数向量$W$点积，即可得出逼近的值函数
$$
\begin{align}
\hat{v}(s,W) & = \left( x_1(s) \ldots x_n(s)\right) · \left( \begin{matrix}
w_1\\
\vdots\\
w_n
\end{matrix}\right)\\
 & = x_1(s)·w_1+\ldots+x_n(s)·w_n\\
 & = \sum_{j=1}^{n}x_j(s)·w_j
\end{align}
$$
4. 线性函数逼近（`Linear Function Approximation`）
   1. 线性函数是所有特征乘以相应的权重并求和的结果
   2. 使用梯度下降的方法调整$W$找到最优参数向量
      1. `Value Function`$\hat{v}(s,W)=X(s)^T·W$，其相对于$W$的导数：$\nabla_W\hat{v}(s,W)=X(s)$
      2. 以平方差的形式计算真值函数$V_\pi$与逼近值函数$\hat{v}$的差异：$$Minimize Error: J(W)=\mathbb{E}_\pi \left[\left(v_\pi(s)-X(s)^T·W\right)^2\right]$$
      3. 使用梯度下降计算该函数相对于$W$的梯度或导数。使用差分链式法则，结果是-2乘以差值乘以$\hat{v}$的导数（也就是特征向量）$$\nabla_WJ(W)=-2\left(v_\pi(s)-X(s)^T·W\right)X(s)$$去掉了期望符号，侧重于单个状态$s$表示的误差梯度
      4. 假设状态是随机选择的，如果能抽取足够多的样本，则可以非常接近预期值，将这个带入梯度下降更新规则的一般公式：$$\begin{align}\Delta W & = -\alpha \frac{1}{2}\nabla_WJ(W)\\& = \alpha\left(v_\pi(s)-X(s)^T·W\right)X(s)\end{align}$$$\alpha$是步长或学习速率参数，$\frac{1}{2}$是为了消去到书中的-2
      5. 将这个基本公式迭代到降低每个样本状态的误差，知道逼近函数和真函数几乎相等
5. 梯度下降优化参数向量的原理：在每次迭代时，朝着误差的相反方向小步地更改权重。其中，特征向量$X(s)$指出了哪个方向不合适。
6. 为了解决不基于模型的控制问题，即在未知环境中采取动作，需要逼近动作值函数。定义一个利用状态和动作的特征转换$$X(s,a)=\left(\begin{matrix}x_1(s,a)\\ x_2(s,a)\\ \vdots \\ x_n(s,a)\end{matrix}\right)$$然后使用状态值函数中用到的梯度下降方法
7. 对于逼近函数同时计算所有动作值的情况：可以看作生成动作向量，可以继续使用相同的特征向量$X(s,a)=\left(\begin{matrix}x_1(s,a)\\ x_2(s,a)\\ \vdots \\ x_n(s,a)\end{matrix}\right)$，传入状态和动作
   - 生成不同的动作值的方法：尝试找到n个不同的动作值函数，每个动作维度对应一个函数，同时计算这些函数，拓展权重向量并转化为矩阵。矩阵的每列模拟一个单独的线性函数，根据状态和动作计算的共同特征，使这些函数相互保持关联性
$$
\begin{align}
\hat{q}(s,a,W) & = \left(x_1(s,a) \ldots x_n(s,a)\right)·\left(
\begin{matrix}
w_{11} & \cdots & w_{1m}\\
\vdots & \ddots & \vdots\\
w_{n1} & \cdots & w_{nm}
\end{matrix}\right)\\
 & = \left(\hat{q}(s,a_1,W) \ldots \hat{q}(s,a_m,W)\right)
\end{align}
$$如果问题领域具有连续状态空间，但具有离散动作空间，并且这是很常见的类型，可以选择值最大的动作。如果动作空间也是连续的，那么这种形式能够同时输出多个值
8. 线性函数逼近的主要限制条件是：只能表示输入和输出之间的线性关系

---
### 4.内核函数
1. 对于特征向量$X(s)=\left(\begin{matrix}x_1(s)\\ x_2(s)\\ \vdots \\ x_n(s)\end{matrix}\right)$中的每一个元素都可以用单独的函数生成，可以是非线性函数，这些函数就是__内核函数__或者__基函数__。他们将输入状态转换成不同的空间，并且因为值函数是这些特征的线性组合，依然可以使用线性函数逼近，使得值函数能够表示输入状态和输出状态之间的非线性关系。径向基函数是常见的用于此目的的内核。
2. 径向基函数$\phi_i(s)\doteq exp\left( -\frac{\Arrowvert s-c_i\Arrowvert^2}{2\sigma_i^2}\right)$
   1. 本质上，将当前状态$s$看作连续空间内的位置，状态空间表示为矩形平面。每个基函数都显示为一个水泡，状态越接近水泡的中心，函数返回值越高。距离越远，返回结果沿半径逐渐减小。
   2. 均值表示水泡中心，标准偏差决定了返回结果降低的平缓程度
   3. 对于任何给定状态，都可以将状态表示法简化为这些径向基函数的返回结果向量，然后使用相同的函数逼近方法

---
### 5.非线性函数逼近（`Non-Linear Function Approximation`）
1. 如果底层的值函数相对这些特征值的组合来说非线性，为了捕获这种复杂的关系，把通过点积获得线性返回结果传入某个非线性函数$f$，也就是最初的激活函数，其大大提高了逼近器的表示能力。$$\hat{v}(s,W)=f\left(X(s)^T·W\right)$$
2. 可以使用梯度下降迭代的更新任何此类函数的参数$\Delta W=\alpha\left(v_\pi(s)-\hat{v}(s,W)\right)\nabla_W\hat{v}(s,W)\tag{梯度下降更新规则}$学习速率$\alpha$乘以值差异乘以函数相对于权重的导数